In [1]:
import numpy as np
import numba as nb
from numba import float64, int64, boolean
import time
import parameters as par

In [2]:
@nb.njit(float64[:](float64, float64, float64[:]))
def Excitation(t, P_amb, args):
    p_A, freq, n = args
    if t < 0.0:
        p_Inf = P_amb
        p_Inf_dot = 0.0
    elif t > n / freq:
        p_Inf = P_amb
        p_Inf_dot = 0.0
    else:
        insin = 2.0*np.pi*freq
        #insin = 2.0*np.pi*freq
        p_Inf = P_amb + p_A*np.sin(insin*t)
        p_Inf_dot = p_A*insin*np.cos(insin*t)

    return np.array([p_Inf, p_Inf_dot], dtype=np.float64)

@nb.njit(float64[:](float64, float64, float64, float64, float64, float64, float64, float64, float64[:]))
def Pressure(t, R, R_dot, mu_L, surfactant, p, p_dot, P_amb, args):
    p_Inf, p_Inf_dot = Excitation(t, P_amb, args)
    p_L = p - (2.0 * surfactant * par.sigma + 4.0 * mu_L * R_dot) / R
    p_L_dot = p_dot + (-2.0 * surfactant * par.sigma * R_dot + 4.0 * mu_L * R_dot ** 2) / (R ** 2)
    delta = (p_L - p_Inf) / par.rho_L
    delta_dot = (p_L_dot - p_Inf_dot) / par.rho_L
    return np.array([delta, delta_dot], dtype=np.float64)

In [3]:
args = np.array([-1e5, 10e3, 1.0], dtype=np.float64)
t = 0.1; R = 1e-6; R_dot = 0.0; mu_L = 1e-3; surfactant = 1.0; p = 1e5; p_dot = 0.0; P_amb = 1e5
%timeit Pressure(t, R, R_dot, mu_L, surfactant, p, p_dot, P_amb, args)
Pressure(t, R, R_dot, mu_L, surfactant, p, p_dot, P_amb, args)

889 ns ± 14.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


array([-144.19955921,    0.        ])

In [4]:
@nb.njit(nb.types.Tuple((float64, float64))(float64, float64, float64[:]))
def Excitation(t, P_amb, args):
    p_A, freq, n = args
    if t < 0.0:
        p_Inf = P_amb
        p_Inf_dot = 0.0
    elif t > n / freq:
        p_Inf = P_amb
        p_Inf_dot = 0.0
    else:
        insin = 2.0*np.pi*freq
        p_Inf = P_amb + p_A*np.sin(insin*t)
        p_Inf_dot = p_A*insin*np.cos(insin*t)

    return p_Inf, p_Inf_dot

@nb.njit(nb.types.Tuple((float64, float64))(float64, float64, float64, float64, float64, float64, float64, float64, float64[:]))
def Pressure(t, R, R_dot, mu_L, surfactant, p, p_dot, P_amb, args):
    p_Inf, p_Inf_dot = Excitation(t, P_amb, args)
    p_L = p - (2.0 * surfactant * par.sigma + 4.0 * mu_L * R_dot) / R
    p_L_dot = p_dot + (-2.0 * surfactant * par.sigma * R_dot + 4.0 * mu_L * R_dot ** 2) / (R ** 2)
    delta = (p_L - p_Inf) / par.rho_L
    delta_dot = (p_L_dot - p_Inf_dot) / par.rho_L
    return delta, delta_dot

%timeit Pressure(t, R, R_dot, mu_L, surfactant, p, p_dot, P_amb, args)
Pressure(t, R, R_dot, mu_L, surfactant, p, p_dot, P_amb, args)

374 ns ± 3.62 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


(-144.19955920657185, 0.0)

In [5]:
def generate(mu_L, surfactant, p, p_dot, P_amb, p_A, freq, n):
    insin = 2.0*np.pi*freq
    n_per_freq = n / freq
    @nb.njit(nb.types.Tuple((nb.float64, nb.float64))(float64))
    def Excitation(t):
        if t < 0.0:
            p_Inf = P_amb
            p_Inf_dot = 0.0
        elif t > n_per_freq:
            p_Inf = P_amb
            p_Inf_dot = 0.0
        else:
            p_Inf = P_amb + p_A*np.sin(insin*t)
            p_Inf_dot = p_A*insin*np.cos(insin*t)

        return p_Inf, p_Inf_dot

    @nb.njit(nb.types.Tuple((nb.float64, nb.float64))(float64, float64, float64, float64, float64))
    def Pressure(t, R, R_dot, p, p_dot):
        p_Inf, p_Inf_dot = Excitation(t)
        p_L = p - (2.0 * surfactant * par.sigma + 4.0 * mu_L * R_dot) / R
        p_L_dot = p_dot + (-2.0 * surfactant * par.sigma * R_dot + 4.0 * mu_L * R_dot ** 2) / (R ** 2)
        delta = (p_L - p_Inf) / par.rho_L
        delta_dot = (p_L_dot - p_Inf_dot) / par.rho_L
        return delta, delta_dot

    return Pressure

%timeit generate(mu_L, surfactant, p, p_dot, P_amb, *args)

138 ms ± 1.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
Pressure = generate(mu_L, surfactant, p, p_dot, P_amb, *args)
%timeit Pressure(t, R, R_dot, p, p_dot)

188 ns ± 0.418 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [7]:
a=np.linspace(1,6,5)

@nb.njit
def f(T):
    T_vec = T ** np.array([0, 1, 2, 3, 4], dtype=np.int64)
    return np.sum(a*T_vec)

print(f(300.0))
%timeit f(300.0)

@nb.njit
def g(T):
    ret = 0.0
    for i in range(5):
        ret += a[i]*T**i
        
    return ret

print(g(300.0))
%timeit g(300.0)

48728565676.0
274 ns ± 8.25 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
48728565676.0
77.9 ns ± 0.574 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [8]:
a=np.array([1.0, 2.0, 3.0, 4.0, 5.0], dtype=np.float64)
b = (1.0, 2.0, 3.0, 4.0, 5.0)

@nb.njit
def f(a):
    return a[0] + a[1] + a[2] + a[3] + a[4]

print(f(a))
%timeit f(a)

@nb.njit
def g(a, b, c, d, e):       
    return a+b+c+d+e

print(g(*b))
%timeit g(*b)

15.0
162 ns ± 1.18 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
15.0
120 ns ± 0.695 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [9]:
@nb.njit
def f():
    a = np.zeros((10), dtype=np.float64)
    b = np.zeros((10), dtype=np.float64)
    c = np.zeros((10), dtype=np.float64)
    d = np.zeros((10), dtype=np.float64)

@nb.njit
def g():
    a = np.zeros((40), dtype=np.float64)

%timeit f()
%timeit g()

264 ns ± 3.07 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
100 ns ± 1.89 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


## Start

In [41]:
def equal(eps=1e-20, **kwargs):
    name1, name2 = kwargs.keys()
    a = kwargs[name1]
    b = kwargs[name2]
    if type(a) == tuple or type(a) == list:
        a = np.array(a)
        b = np.array(b)
    if type(a) == np.ndarray:
        if type(a[0]) == np.ndarray:
            ret = True
            for i, (aa, bb) in enumerate(zip(a, b)):
                ret = ret and equal(eps, **{name1+str(i): aa, name2+str(i): bb})
            return ret
        else:
            diff = np.abs(a - b) / (abs(b) + 1e-30)
            wrong = np.where(diff > eps)[0]
            if len(wrong) > 0:
                print(colored(f'{name1} != {name2} at indexes:', 'red'), wrong)
                for i in wrong:
                    print(f'\t{i}: {a[i]: e} != {b[i]: e}\t (diff: {diff[i]: e})')
                return False
            return True
    else:
        diff = abs(a - b) / (abs(b) + 1e-30)
        if diff > eps:
            print(colored(f'{name1} != {name2}', 'red'))
            print(f'\t{a: e} != {b: e}\t (diff: {diff: e})')
            return False
        return True

In [11]:
"""Create parameters.py and load it"""

# Directory for .inp file:
path = 'D:/parameter_studies/Bubble_dynamics_simulation/INP file examples/chem_Otomo2018_without_O.inp'

# import libraries:
import importlib   # For reloading your own files
import numpy as np
# my own files:
import Bubble_dynamics_simulation.inp_data_extractor as inp   # numeric constants and coefficents
importlib.reload(inp)   # reload changes you made
inp.extract(path)

import parameters as par   # numeric constants and coefficents
importlib.reload(par)   # reload changes you made
print(par.model)

path=D:/parameter_studies/Bubble_dynamics_simulation/INP file examples/chem_Otomo2018_without_O.inp
Note, lambda value for specie 'H' is not in data.py: 0.0 is used
Note, lambda value for specie 'NH2' is not in data.py: 0.0 is used
Note, lambda value for specie 'NH' is not in data.py: 0.0 is used
Note, lambda value for specie 'N' is not in data.py: 0.0 is used
Note, lambda value for specie 'NNH' is not in data.py: 0.0 is used
Note, lambda value for specie 'N2H4' is not in data.py: 0.0 is used
Note, lambda value for specie 'N2H3' is not in data.py: 0.0 is used
Note, lambda value for specie 'N2H2' is not in data.py: 0.0 is used
Note, lambda value for specie 'H2NN' is not in data.py: 0.0 is used
Warning, third body 'H2O' is not in species in line 64 (' H2/2.5/ H2O/12/ AR/0.0/ ') in reaction 'H2+M=H+H+M'
Warning, third body 'AR' is not in species in line 64 (' H2/2.5/ H2O/12/ AR/0.0/ ') in reaction 'H2+M=H+H+M'
model: chem_Otomo2018_without_O
File 'parameters.py' succesfully created
chem_O

In [12]:
from Bubble_dynamics_simulation import full_bubble_model_old as de

model: chem_Otomo2018_without_O
target specie: NH3
excitation: sin_impulse (control parameters: ['p_A', 'freq', 'n'])
enable heat transfer: True	enable evaporation: False	enable reactions: True	enable dissipated energy: True


In [13]:
from termcolor import colored
import matplotlib.pyplot as plt   # for plotting
import numpy as np   # matrices, math
from scipy.integrate import solve_ivp   # differential equation solver
from scipy.signal import argrelmin   # loc min finding
import time   # runtime measurement
from datetime import datetime   # for accessing current datetime
import socket   # for accessing computer name
import psutil   # get system information
from numba import njit   # Just In Time compiler
from numba.types import Tuple, unicode_type, float64, float32, int64, int32   # JIT types
from func_timeout import func_timeout, FunctionTimedOut   # for timeout
import os    # file management
import importlib   # for reloading your own files

# my own files:
try:
    import parameters as par   # numeric constants and coefficents
    importlib.reload(par)   # reload changes you made
except:
    print(print(colored('Error, \'parameters.py\' not found','red')))
try:
    import excitation
    importlib.reload(excitation)
except:
    try:
        import Bubble_dynamics_simulation.excitation as excitation
        importlib.reload(excitation)
    except:
        print(colored(f'Error, \'excitation.py\' not found', 'red'))

enable_heat_transfer = True
enable_evaporation = False
enable_reactions = True
enable_dissipated_energy = True
target_specie = 'NH3' # Specie to calculate energy effiqiency
excitation_type = 'sin_impulse' # function to calculate pressure excitation

Excitation, excitation_args, excitation_units = excitation.getExcitation(excitation_type=excitation_type)

In [16]:
t = 4.26797748831401e-05
x = np.array([2.77682627e-07, 7.09472642e+02, 1.00722324e+04, 1.77061349e-01,
       7.84400226e-01, 1.03611485e-01, 4.05593217e-01, 5.37429074e-02,
       5.76856803e-03, 4.65051672e-02, 3.29928678e-02, 2.11635072e-02,
       4.63225690e-03, 3.13284401e-03, 1.24827451e-01, 2.05414031e-05], dtype=np.float64)

In [17]:
@njit(Tuple((float64, float64))(float64, float64, float64, float64, float64, float64, float64, float64, float64[:]))
def Pressure(t, R, R_dot, mu_L, surfactant, p, p_dot, P_amb, args):
    (p_Inf, p_Inf_dot) = Excitation(t, P_amb, args)
    p_L = p - (2.0 * surfactant * par.sigma + 4.0 * mu_L * R_dot) / R
    p_L_dot = p_dot + (-2.0 * surfactant * par.sigma * R_dot + 4.0 * mu_L * R_dot ** 2) / (R ** 2)
    delta = (p_L - p_Inf) / par.rho_L
    delta_dot = (p_L_dot - p_Inf_dot) / par.rho_L
    return delta, delta_dot

%timeit Pressure(t, 10e-6, 0.0, 0.001, 1.0, 1e5, 0.0, 1e5, args)
%timeit de.Pressure(t, 10e-6, 0.0, 0.001, 1.0, 1e5, 0.0, 1e5, args)
equal(new=Pressure(t, 10e-6, 0.0, 0.001, 1.0, 1e5, 0.0, 1e5, args), old=de.Pressure(t, 10e-6, 0.0, 0.001, 1.0, 1e5, 0.0, 1e5, args))

378 ns ± 0.526 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
899 ns ± 0.799 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [170]:
# returns molar heat capacities, enthalpies and entropies
@njit(float64[:, :](float64))
def Thermodynamic(T):
    ret = np.zeros((4, par.K), dtype=np.float64)   # [C_p, H, S, C_v]
    for k in range(par.K):
    # get coefficients for T
        if T <= par.TempRange[k][2]: # T <= T_mid
            a = par.a_low[k]
        else:  # T_mid < T
            a = par.a_high[k]
    # calculate sums
        C_p = H = S = 0.0
        for n in range(par.N): # [0, 1, 2, 3, 4]
            T_pow = T**n
            C_p += a[n] * T_pow
            H += a[n] * T_pow / float64(n+1)
            if n != 0:
                S += a[n] * T_pow / float64(n)
    # calculations outside the sums
        # Molar heat capacities at constant pressure (isobaric) [erg/mol/K]
        ret[0][k] = par.R_erg * C_p
        # Enthalpies [erg/mol]
        ret[1][k] = par.R_erg * (T * H + a[par.N])
        # Entropies [erg/mol/K]
        ret[2][k] = par.R_erg * (a[0] * np.log(T) + S + a[par.N+1])
        # Molar heat capacities at constant volume (isochoric) [erg/mol/K]
        ret[3][k] = ret[0][k] - par.R_erg

    return ret

In [171]:
T = 10102.835510879857
%timeit Thermodynamic(T)
c = x[3:3+par.K]
%timeit de.Thermodynamic(c, T)
equal(new=Thermodynamic(T), old=de.Thermodynamic(c, T), eps=1e-100)

573 ns ± 7.79 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
4.43 µs ± 19.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


True

In [23]:
@njit(Tuple((float64, float64))(float64, float64, float64, float64, float64, float64))
def Evaporation(p, T, X_H2O, alfa_M, T_inf, P_v):
# condensation and evaporation
    p_H2O = X_H2O * p
    n_eva_dot = 1.0e3 * alfa_M * P_v / (par.W[par.indexOfWater] * np.sqrt(2.0 * np.pi * par.R_v * T_inf))
    n_con_dot = 1.0e3 * alfa_M * p_H2O / (par.W[par.indexOfWater] * np.sqrt(2.0 * np.pi * par.R_v * T))
    n_net_dot = n_eva_dot - n_con_dot
# Molar heat capacity of water at constant volume (isochoric) [J/mol/K]
    # get coefficients for T
    if T <= par.TempRange[par.indexOfWater][2]: # T <= T_mid
        a = par.a_low[par.indexOfWater]
    else:  # T_mid < T
        a = par.a_high[par.indexOfWater]
    # calculate sum
    T_pow = 1.0
    C_V = 0.0
    for n in range(1, par.N+1): # [1, 2, 3, 4, 5]
        C_V += a[n-1] * T_pow
        T_pow *= T
    C_V = par.R_erg * (C_V - 1.0)
    
    # get coefficients for T
    if T_inf <= par.TempRange[par.indexOfWater][2]: # T_inf <= T_mid
        a = par.a_low[par.indexOfWater]
    else:  # T_mid < T_inf
        a = par.a_high[par.indexOfWater]
    # calculate sum
    T_pow = 1.0
    C_V_inf = 0.0
    for n in range(1, par.N+1): # [1, 2, 3, 4, 5]
        C_V_inf += a[n-1] * T_pow
        T_pow *= T_inf
    C_V_inf = par.R_erg * (C_V_inf - 1.0)
# Evaporation energy [J/mol]
    e_eva = C_V_inf * T_inf * 1e-7
    e_con = C_V * T * 1e-7
    evap_energy = n_eva_dot * e_eva - n_con_dot * e_con    # [W/m^2]
    
    return n_net_dot, evap_energy

In [24]:
M = np.sum(c); X_H2O = c[par.indexOfWater]/M; alfa_M = 0.35; T_inf = 300.0; P_v = par.P_v
p = 0.1 * M * par.R_erg * T
C_p, H, S, C_v = Thermodynamic(T)
%timeit Evaporation(p, T, X_H2O, alfa_M, T_inf, P_v)
%timeit de.Evaporation(M, T, X_H2O, alfa_M, T_inf, P_v)
equal(old=Evaporation(p, T, X_H2O, alfa_M, T_inf, P_v), new=de.Evaporation(M, T, X_H2O, alfa_M, T_inf, P_v), eps=1e-100)

289 ns ± 3.36 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
1.26 µs ± 6.23 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [25]:
@njit(float64[:](float64, float64[:], float64, float64))
def ForwardRate(T, M_eff, M, p):
# Reaction rate
    k_forward = par.A * T ** par.b * np.exp(-par.E / (par.R_cal * T))
    
# Pressure dependent reactions
    for j, i in enumerate(par.PressureDependentIndexes):    # i is the number of reaction, j is the index of i's place in par.PressureDependentIndexes
        k_inf = k_forward[i]    # par.A[i] * T ** par.b[i] * np.exp(-par.E[i] / (par.R_cal * T))
        k_0 = par.ReacConst[j][0] * T ** par.ReacConst[j][1] * np.exp(-par.ReacConst[j][2] / (par.R_cal * T))
        if i in par.ThirdBodyIndexes:
            thirdBodyIndex = 0
            while par.ThirdBodyIndexes[thirdBodyIndex] != i:
                thirdBodyIndex += 1
            M_eff_loc = M_eff[thirdBodyIndex]
            thirdBodyIndex += 1
        else:
            M_eff_loc = M
        P_r = k_0 / k_inf * M_eff_loc
        
        # Lindemann formalism
        if i in par.LindemannIndexes:
            F = 1.0

        # Troe formalism
        elif i in par.TroeIndexes:
            F_cent = (1.0 - par.Troe[j][0]) * np.exp(-T / par.Troe[j][1]) + par.Troe[j][0] * np.exp(-T / par.Troe[j][2]) + np.exp(-par.Troe[j][3] / T)
            logF_cent = np.log10(F_cent)
            c2 = -0.4 - 0.67 * logF_cent
            n = 0.75 - 1.27 * logF_cent
            d = 0.14
            logP_r = np.log10(P_r)
            logF = 1.0 / (1.0 + ((logP_r + c2) / (n - d * (logP_r + c2))) ** 2) * logF_cent
            F = 10.0 ** logF
        
        # SRI formalism
        elif i in par.SRIIndexes: 
            X = 1.0 / (1.0 + np.log10(P_r)**2)
            F = par.SRI[j][3] * (par.SRI[j][0] * np.exp(-par.SRI[j][1] / T) + np.exp(-T / par.SRI[j][2]))**X * T ** par.SRI[j][4]
    # Pressure dependent reactions END
    
        k_forward[i] = k_inf * P_r / (1.0 + P_r) * F

# PLOG reactions
    for j, i in enumerate(par.PlogIndexes):
        if p < par.Plog[3*j+1][0]:
            k_1 = par.Plog[3*j][1] * T ** par.Plog[3*j][2] * np.exp(-par.Plog[3*j][3] / (par.R_cal * T))
            k_2 = par.Plog[3*j+1][1] * T ** par.Plog[3*j+1][2] * np.exp(-par.Plog[3*j+1][3] / (par.R_cal * T))
            ln_k = np.log(k_1) + (np.log(p) - np.log(par.Plog[3*j][0])) / (np.log(par.Plog[3*j+1][0]) - np.log(par.Plog[3*j][0])) * (np.log(k_2) - np.log(k_1))
            k_forward[i] = np.exp(ln_k)
        else:
            k_2 = par.Plog[3*j+1][1] * T ** par.Plog[3*j+1][2] * np.exp(-par.Plog[3*j+1][3] / (par.R_cal * T))
            k_3 = par.Plog[3*j+2][1] * T ** par.Plog[3*j+2][2] * np.exp(-par.Plog[3*j+2][3] / (par.R_cal * T))
            ln_k = np.log(k_2) + (np.log(p) - np.log(par.Plog[3*j+1][0])) / (np.log(par.Plog[3*j+2][0]) - np.log(par.Plog[3*j+1][0])) * (np.log(k_3) - np.log(k_2))
            k_forward[i] = np.exp(ln_k)
            
    return k_forward

In [26]:
M_eff1 = np.zeros((par.ThirdBodyCount), dtype = np.float64)   # effective total concentration of the third-body 
for j, i in enumerate(par.ThirdBodyIndexes):
    M_eff1[j] = np.sum(par.alfa[j] * c)
M_eff2 = M * np.ones((par.I), dtype = np.float64)    # effective total concentration of the third-body 
for j, i in enumerate(par.ThirdBodyIndexes):
    M_eff2[i] = np.sum(par.alfa[j] * c) 
%timeit ForwardRate(T, M_eff1, M, p)
%timeit de.ForwardRate(T, M_eff2, M)
equal(new=ForwardRate(T, M_eff1, M, p), old=de.ForwardRate(T, M_eff2, M), eps=1e-100)

1.76 µs ± 8.44 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
1.75 µs ± 14.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [30]:
@njit(float64[:](float64[:], float64[:], float64[:], float64, float64))
def BackwardRate(k_forward, S, H, T, P_amb):
    k_backward = np.zeros((par.I), dtype=np.float64)
    for i in range(par.I):
        DeltaS = 0.0
        DeltaH = 0.0
        for k in range(par.K):
            DeltaS += par.nu[i][k] * S[k]
            DeltaH += par.nu[i][k] * H[k]
        K_p = np.exp(DeltaS / par.R_erg - DeltaH / (par.R_erg * T))
        K_c = K_p * (P_amb * 10.0 / (par.R_erg * T)) ** np.sum(par.nu[i])
        k_backward[i] = k_forward[i] / K_c
    for i in par.IrreversibleIndexes:
        k_backward[i] = 0.0
        
    return k_backward

k_forward = ForwardRate(T, M_eff1, M, p)
%timeit BackwardRate(k_forward, S, H, T, P_amb)
%timeit de.BackwardRate(k_forward, S, H, T, P_amb)
equal(new=BackwardRate(k_forward, S, H, T, P_amb), old=de.BackwardRate(k_forward, S, H, T, P_amb), eps=1e-100)

1.56 µs ± 10.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
3.46 µs ± 26.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [31]:
@njit(float64[:](float64, float64[:], float64[:], float64[:], float64, float64, float64))
def ProductionRate(T, H, S, c, P_amb, p, M):
# Third body correction factors
    M_eff = np.zeros((par.ThirdBodyCount), dtype = np.float64)   # effective total concentration of the third-body 
    for j, i in enumerate(par.ThirdBodyIndexes):
        for k in range(par.K):
            M_eff[j] += par.alfa[j][k] * c[k]
# Forward and backward rates
    k_forward = ForwardRate(T=T, M_eff=M_eff, M=M, p=p)
    k_backward = BackwardRate(k_forward=k_forward, S=S, H=H, T=T, P_amb=P_amb)

# Net rates
    q = np.zeros((par.I), dtype = np.float64)
    for i in range(par.I):
        forward = 1.0
        backward = 1.0
        for k in range(par.K):
            forward *= c[k] ** par.nu_forward[i][k]
            backward *= c[k] ** par.nu_backward[i][k]
        q[i] = k_forward[i] * forward - k_backward[i] * backward
# Third body reactions
    for j, i in enumerate(par.ThirdBodyIndexes):    # i is the number of reaction, j is the index of i in par.ThirdBodyIndexes
        if i not in par.PressureDependentIndexes:
            q[i] *= M_eff[j]
# Production rates
    omega_dot = np.zeros((par.K), dtype=np.float64)
    for k in range(par.K):
        for i in range(par.I):
            omega_dot[k] += par.nu[i, k] * q[i]
    
    return omega_dot

%timeit ProductionRate(T, H, S, c, P_amb, p, M)
%timeit de.ProductionRate(T, H, S, c, P_amb, M)
equal(new=ProductionRate(T, H, S, c, P_amb, p, M), old=de.ProductionRate(T, H, S, c, P_amb, M), eps=1e-100)

6.13 µs ± 51.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
13 µs ± 158 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [34]:
kwargs = dict(t=t, x=x, P_amb=1e5, alfa_M=0.35, T_inf=300.0, surfactant=1.0, P_v=par.P_v, mu_L=par.mu_L, c_L=par.c_L, ex_args=args)

In [169]:
@njit(float64[:](float64, float64[:], float64, float64, float64, float64, float64, float64, float64, float64[:]))
def f(t, x, P_amb, alfa_M, T_inf, surfactant, P_v, mu_L, c_L, ex_args):   
    R = x[0]      # bubble radius [m]
    R_dot = x[1]  # [m/s]
    T = x[2]      # temperature [K]
    c = x[3:-1]     # molar concentration [mol/cm^3]
    M = np.sum(c) # sum of concentration
    X = c / M     # mole fraction [-]
    p = 0.1 * M * par.R_erg * T # Partial pressure of the gases [Pa]
    dxdt = np.zeros(x.shape, dtype = np.float64)
    
# d/dt R
    dxdt[0] = R_dot
# Thermodynamics
    (C_p, H, S, C_v) = Thermodynamic(T=T)
    W_avg = C_p_avg = C_v_avg = lambda_avg = 0.0
    for k in range(par.K):
        W_avg += X[k] * par.W[k]
        C_p_avg += X[k] * C_p[k]
        C_v_avg += X[k] * C_v[k]
        lambda_avg += X[k] * par.lambdas[k]

    if enable_heat_transfer:
        rho_avg = W_avg * M # or np.sum(c * par.W)
        chi_avg = 10.0 * lambda_avg * W_avg / (C_p_avg * rho_avg)
        l_th = np.inf
        if R_dot != 0.0:
            l_th = np.sqrt(R * chi_avg / abs(R_dot))
        l_th = min(l_th, R / np.pi)
        Q_th_dot = lambda_avg * (T_inf - T) / l_th
    else:
        Q_th_dot = 0.0
# d/dt c
    if enable_reactions:
        omega_dot = ProductionRate(T=T, H=H, S=S, c=c, P_amb=P_amb, p=p, M=M)
    else:
        omega_dot = np.zeros((par.K), dtype = np.float64)
    c_dot = omega_dot - c * 3.0 * R_dot / R
# Evaporation
    if enable_evaporation:
        n_net_dot, evap_energy = Evaporation(p=p, T=T, X_H2O=X[par.indexOfWater], alfa_M=alfa_M, T_inf=T_inf, P_v=P_v)
        c_dot[par.indexOfWater] += 1.0e-6 * n_net_dot * 3.0 / R    # water evaporation
    else:
        n_net_dot = evap_energy = 0.0
    dxdt[3:-1] = c_dot
# d/dt T
    sum_omega_dot = np.sum(omega_dot)
    Q_r_dot = 0.0
    for k in range(par.K):
        Q_r_dot -= omega_dot[k] * H[k]
    Q_r_dot += sum_omega_dot * par.R_erg * T
    T_dot = (Q_r_dot + 30.0 / R * (-p * R_dot + Q_th_dot + evap_energy)) / (M * C_v_avg)
    p_dot = p * (sum_omega_dot / M + T_dot / T - 3.0 * R_dot / R) # for later use
    dxdt[2] = T_dot
# d/dt R_dot
    (delta, delta_dot) = Pressure(t=t,
        R=R, R_dot=R_dot, mu_L=mu_L, surfactant=surfactant,
        p=p, p_dot=p_dot, P_amb=P_amb, args=ex_args
    )   # delta = (p_L-P_amb) / rho_L
    
    Nom = (1.0 + R_dot / c_L) * delta + R / c_L * delta_dot - (1.5 - 0.5 * R_dot / c_L) * R_dot ** 2
    Den = (1.0 - R_dot / c_L) * R + 4.0 * mu_L / (c_L * par.rho_L)
    
    dxdt[1] = Nom / Den
    
    if enable_dissipated_energy:
        V_dot=4.0 * R * R * R_dot * np.pi
        integrand_th = -(p * (1 + R_dot / c_L) + R / c_L * p_dot) * V_dot
        integrand_v = 16.0 * np.pi * mu_L * (R * R_dot*R_dot + R * R * R_dot * dxdt[1] / c_L)
        integrand_r = 4.0 * np.pi / c_L * R * R * R_dot * (R_dot * p + p_dot * R - 0.5 * par.rho_L * R_dot * R_dot * R_dot - par.rho_L * R * R_dot * dxdt[1])

        dxdt[-1]=(integrand_th + integrand_v + integrand_r)
    else:
        dxdt[-1]=0.0
    
    return dxdt

%timeit f(**kwargs)
%timeit de.f(**kwargs)
equal(new=f(**kwargs), old=de.f(**kwargs), eps=1e-100)

6.79 µs ± 49 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
18.8 µs ± 190 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


True

In [36]:
equal(new=f(**kwargs), old=de.f(**kwargs), eps=1e-100)